In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path


In [ ]:
df = pd.read_parquet(Path("../reports/publicsplit.parquet"))
print(len(df))
df["acc_diff"] = df["Acc1"] - df["Acc2"]
df.head()

Only needed if you want to build the full table including results on CPU and with fixed seeds.
Otherwise, skip the following cells (until the next section).

In [ ]:

cpu = pd.read_parquet(Path("../reports/public_gatcpu.parquet"))
cpu.loc[:, "Model"] = "GAT (CPU)"
cpu["acc_diff"] = cpu["Acc1"] - cpu["Acc2"]
cpu.head()

In [ ]:
df = df.append(cpu, ignore_index=True)

In [ ]:
fixedseed = pd.read_parquet("../reports/public_fixedseed.parquet")


def convert_name(s: str):
    if s == "GAT2017":
        return "GAT (fixed)"
    elif s == "GCN2017":
        return "GCN (fixed)"
    else:
        return s

fixedseed.loc[:, "Model"] = fixedseed["Model"].apply(convert_name)
fixedseed["acc_diff"] = fixedseed["Acc1"] - fixedseed["Acc2"]
fixedseed.head()

In [ ]:
df = df.append(fixedseed, ignore_index=True)

In [ ]:

fixedseed_cpu = pd.read_parquet("/root/feature-similarity/reports/public_fixedseed_cpu.parquet")


def convert_name(s: str):
    if s == "GAT2017":
        return "GAT (fixed, CPU)"
    elif s == "GCN2017":
        return "GCN (fixed, CPU)"
    else:
        return s

fixedseed_cpu.loc[:, "Model"] = fixedseed_cpu["Model"].apply(convert_name)
fixedseed_cpu["acc_diff"] = fixedseed_cpu["Acc1"] - fixedseed_cpu["Acc2"]
fixedseed_cpu.head()

In [ ]:
df = df.append(fixedseed_cpu, ignore_index=True)

## Create a summary table

In [ ]:
meantab = pd.pivot_table(df, values="Value", index=["Dataset", "Model"], columns=["Metric"], aggfunc=lambda x: 100 * np.mean(x))
stdtab = pd.pivot_table(df, values="Value", index=["Dataset", "Model"], columns=["Metric"], aggfunc=lambda x: 100 * np.std(x))
meanacctab = pd.pivot_table(df, values="Acc-mean", index=["Dataset", "Model"], aggfunc=lambda x: 100 * np.mean(x))
stdacctab = pd.pivot_table(df, values="Acc-std", index=["Dataset", "Model"], aggfunc=lambda x: 100 * np.mean(x))

In [ ]:
mean_conv = lambda x: f"{x:0.1f} $\\pm$"
std_conv = lambda x: f" {x:0.1f}"
tab = meantab.applymap(mean_conv) + stdtab.applymap(std_conv)
tempseries = meanacctab.applymap(mean_conv)["Acc-mean"] + stdacctab.applymap(std_conv)["Acc-std"]
tempseries.name = "Acc-mean"
tab = tab.join(tempseries)


def col_order(idx):
    col_to_idx = {
        "Acc-mean": 0,
        "PI": 1,
        "NormPI": 2,
        "True PI": 3,
        "False PI": 4,
        "MAE": 5,
        "SymKL": 6,
    }
    return pd.Index([col_to_idx[s] for s in idx])


def row_order(idx):
    col_to_idx = {
        "CiteSeer": 0,
        "Pubmed": 1,
        "CS": 2,
        "Physics": 3,
        "Computers": 4,
        "Photo": 5,
        "WikiCS": 6,
    }
    return pd.Index([col_to_idx[s] for s in idx])


tab = tab.sort_index(axis=1, key=col_order)
tab = tab.sort_index(axis=0, key=row_order, level=0)


col_to_name = {
    "Acc-mean": "Accuracy",
    "PI": "$d$",
    "NormPI": "$d_{norm}$",
    "True PI": "$d_{True}$",
    "False PI": "$d_{False}$",
    "MAE": "MAE",
    "SymKL": "SymKL",
}
tab = tab.rename(col_to_name, axis="columns")
model_to_name = {"GAT2017": "GAT", "GCN2017": "GCN"}
tab = tab.rename(model_to_name, axis="rows", level=1)


tab = tab.drop("SymKL", axis="columns")
tab
tab.to_latex(
    "../reports/tab1.tex",
    sparsify=True,
    escape=False,
    multirow=True,
    multicolumn=False,
    column_format="llcccccc",
)

